## Pre-screening

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import metrics
from time import time

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [2]:
customers = pd.read_csv('1 - Project Data.csv')

#### Data Inspection
- I'm assuming Churn Value is our label
    - If they did leave 1, didn't 0
- What needs to be changed?
    - Country
    - State
    - City
    - Zip Code?
    - Geographic info
    - Gender (map binary - any non-binary gender data?)
    - Streaming TV (map binary)
    - Streaming Movies (map binary)
    - Payment method (OHE)
    - Churn Label (drop), we already have churn value
        - Is churn value binary?
    - Churn reason?
        - Nulls, drop after encoding

In [11]:
customers.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices


In [12]:
len(customers['Zip Code'].unique())

1652

In [23]:
group = customers.groupby(['City'])['Zip Code'].nunique()

In [24]:
zipcodes = group.reset_index()

In [25]:
zipcodes

,City,Zip Code
0,Acampo,1
1,Acton,1
2,Adelanto,1
3,Adin,1
4,Agoura Hills,1
...,...,...
1124,Yreka,1
1125,Yuba City,2
1126,Yucaipa,1
1127,Yucca Valley,1


In [32]:
zipcodes = zipcodes.sort_values(by=['Zip Code'],ascending=False)

In [33]:
zipcodes.head()

,City,Zip Code
562,Los Angeles,61
869,San Diego,30
878,San Jose,28
855,Sacramento,27
872,San Francisco,26


In [31]:
zipcodes[zipcodes['City'] == 'Zenia']

,City,Zip Code
1128,Zenia,1


In [19]:
len(customers['City'].unique())

1129

In [20]:
customers.iloc[0]

CustomerID                             3668-QPYBK
Count                                           1
Country                             United States
State                                  California
City                                  Los Angeles
Zip Code                                    90003
Lat Long                   33.964131, -118.272783
Latitude                                33.964131
Longitude                             -118.272783
Gender                                       Male
Senior Citizen                                 No
Partner                                        No
Dependents                                     No
Tenure Months                                   2
Phone Service                                 Yes
Multiple Lines                                 No
Internet Service                              DSL
Online Security                               Yes
Online Backup                                 Yes
Device Protection                              No


In [21]:
set(customers['Gender'])

{'Female', 'Male'}

In [22]:
set(customers['Phone Service'])

{'No', 'Yes'}

In [5]:
customers['Phone Service'].isnull().sum()

0

In [6]:
# if they don't have internet, they won't have these!
set(customers['Internet Service'])

{'DSL', 'Fiber optic', 'No'}

In [7]:
# there is only phone and internet so if they don't have phone service, they don't have multiple lines.
set(customers['Multiple Lines'])

{'No', 'No phone service', 'Yes'}

In [8]:
# we need a training set that balances these categories 50/50
set(customers['Churn Reason'])

{'Attitude of service provider',
 'Attitude of support person',
 'Competitor had better devices',
 'Competitor made better offer',
 'Competitor offered higher download speeds',
 'Competitor offered more data',
 'Deceased',
 "Don't know",
 'Extra data charges',
 'Lack of affordable download/upload speed',
 'Lack of self-service on Website',
 'Limited range of services',
 'Long distance charges',
 'Moved',
 'Network reliability',
 'Poor expertise of online support',
 'Poor expertise of phone support',
 'Price too high',
 'Product dissatisfaction',
 'Service dissatisfaction',
 nan}

In [9]:
#call for bootstrapping
customers['Churn Value'].value_counts()

0    5174
1    1869
Name: Churn Value, dtype: int64

In [10]:
customers['Churn Reason'].value_counts()

Attitude of support person                   192
Competitor offered higher download speeds    189
Competitor offered more data                 162
Don't know                                   154
Competitor made better offer                 140
Attitude of service provider                 135
Competitor had better devices                130
Network reliability                          103
Product dissatisfaction                      102
Price too high                                98
Service dissatisfaction                       89
Lack of self-service on Website               88
Extra data charges                            57
Moved                                         53
Limited range of services                     44
Long distance charges                         44
Lack of affordable download/upload speed      44
Poor expertise of phone support               20
Poor expertise of online support              19
Deceased                                       6
Name: Churn Reason, 

Feature engineering
- Remove customer ID for generating the model
- State and Country is the same so can be removed
- City is massive.

## Log Reg

In [35]:
## Logistic Regression
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [36]:
df = pd.read_csv("1 - Project Data.csv")

In [ ]:
##Data Cleaning

In [ ]:
##Feature Engineering

In [ ]:
df_model = 

#### Model Building

In [ ]:
lr = LogisticRegression(max_iter=300, random_state=10)

In [ ]:
lr.fit(df_model, y)

In [ ]:
lr.predict_proba(df_model)

In [ ]:
df_model[['??', '???']] = lr.predict_proba(df_model)

In [ ]:
df_model = df[Features].copy()

### Performance

In [ ]:
print(metrics.confusion_matrix(y, df_model['y_pred']))

In [51]:
def get_results(actual, predicted):
    print("The confusion matrix for your predictions is:")
    print(metrics.confusion_matrix(actual, predicted), "\n")
    print(f'The accuracy of your model is: {metrics.accuracy_score(actual, predicted)}')
    print(f'The recall of your model is: {metrics.recall_score(actual, predicted)}')
    print(f'The precision of your model is: {metrics.precision_score(actual, predicted)}')
    print(f'The F1-score of your model is: {metrics.f1_score(actual, predicted)}')

In [ ]:
get_results(y, df_model['y_pred'])